Trial Targeting Workflow
1) Import and replace DB versions of the shopping log and member contacts table( CIVI reports **"Constituent Summary"** and **"Shopping Log - Activity Report"**)
2) run the trial_shopping .sql/stored procedure that selects for targets
3) extract the result set that contains contact info and shopping stats

Place raw CIVI report extracts into **/membershipReportsCIVI/membershipReportingLogicSampleReports/**
NOTE: the stored procedure contains all the logic to calculate avg trips per cohort, and *currently* DOES NOT apply "leave out current trial member" when calculating avg trips 

In [2]:
# module contains some very basic pd. data frame processing
import sqlalchemy
import pandas as pd
import datetime
import sys
import importlib

In [3]:
import sys
sys.path

['/home/candela/Documents/greeneHill/membershipReportsCIVI/github/greeneHill',
 '/home/candela/anaconda3/envs/py_312/lib/python312.zip',
 '/home/candela/anaconda3/envs/py_312/lib/python3.12',
 '/home/candela/anaconda3/envs/py_312/lib/python3.12/lib-dynload',
 '',
 '/home/candela/anaconda3/envs/py_312/lib/python3.12/site-packages']

In [4]:
import fileProcessing2
importlib.reload(fileProcessing2)
from fileProcessing2 import process_shoplog, process_contacts

In [5]:
shop = process_shoplog('/home/candela/Documents/greeneHill/membershipReportsCIVI/member_shopping/civi_member_shopping/memberShopping_20240722.csv')
contacts = process_contacts('/home/candela/Documents/greeneHill/membershipReportsCIVI/membershipReportingLogicSampleReports/constituentSummaryReport_20240722.csv')

In [6]:
# DEFINE THE DATABASE CREDENTIALS
user = 'root'
password = 'baeldung'
host = '172.17.0.2'
port = 3306
database = 'membership'

def get_connection():
	return sqlalchemy.create_engine(
		url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
			user, password, host, port, database
		)
	)

if __name__ == '__main__':

	try:
	
		# GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
		engine = get_connection()
		print(
			f"Connection to the {host} for user {user} created successfully.")
	except Exception as ex:
		print("Connection could not be made due to the following error: \n", ex)


Connection to the 172.17.0.2 for user root created successfully.


In [7]:
#db injection
with engine.connect() as conn:
    shop.loc[:,['Target_Name', 'Target_Email', 'Activity_Type', 'Subject', 'Activity_Date','ingest_date']].to_sql('shop_log', 
    con=engine, if_exists='replace', index=False)

    contacts.to_sql('member_directory2', con=engine, if_exists='replace', index=False)

Call the stored procedure. Ultimately creates a temporary table **trial_tgts** with the period's target individuals

In [8]:
from sqlalchemy import text
#a Core approach (because I'm interacting explicitly w/the engine as opposed to abstracted objects), where I write explicit SQL code
#copied code from https://docs.sqlalchemy.org/en/20/core/connections.html
connection = engine.raw_connection()
try:
    cursor_obj = connection.cursor()
    cursor_obj.callproc("trial_targets") #./stored_procedure_trial_shopping.sql
    cursor_obj.close()
    connection.commit()
finally:
        connection.close()


In [9]:
sql = "SELECT * FROM trial_tgts"
df = pd.read_sql(sql, con=engine)

In [10]:
df.shape

(28, 10)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   FirstDayOfWeek    28 non-null     object 
 1   email             28 non-null     object 
 2   trips             28 non-null     int64  
 3   trial_expiration  28 non-null     object 
 4   orig_type_clean   28 non-null     object 
 5   cohort_avg        28 non-null     float64
 6   contact_name      28 non-null     object 
 7   first_name        28 non-null     object 
 8   last_name         28 non-null     object 
 9   phone             28 non-null     object 
dtypes: float64(1), int64(1), object(8)
memory usage: 2.3+ KB


In [12]:
df.head()

,FirstDayOfWeek,email,trips,trial_expiration,orig_type_clean,cohort_avg,contact_name,first_name,last_name,phone
0,2024-01-14,sworleyj@gmail.com,14,2024-07-15,6 mo trial,2.7273,"Worley, Spencer",Spencer,Worley,3368130546
1,2024-01-21,jaywhysel@gmail.com,16,2024-07-19,6 mo trial,3.2857,"Whysel, Josephine",Josephine,Whysel,16469426917
2,2024-01-28,stephanie.reda2@gmail.com,8,2024-07-26,6 mo trial,5.6667,"Reda, Stephanie",Stephanie,Reda,9142752618
3,2024-01-28,gpiedmonte@gmail.com,19,2024-08-01,6 mo trial,5.6667,"Piedmonte, Gabrielle",Gabrielle,Piedmonte,646.345.6235
4,2024-01-28,natgable@gmail.com,14,2024-07-30,6 mo trial,5.6667,"Gable, Natalie",Natalie,Gable,4155316497


In [13]:
#clean up some columns headers
df.rename(columns = {'FirstDayOfWeek':'week_of_trial_start','orig_type_clean':'trial_type'},inplace = True)

In [14]:
filepath = '/home/candela/Documents/greeneHill/membershipReportsCIVI/trial_target_roster/'
filename = 'trial_tgts_'+datetime.datetime.strftime(datetime.date.today(),format="%Y-%m-%d")


In [15]:

df.to_csv(filepath+filename+'.csv',index = False)